# Prerocessing Data


In [24]:
import pandas as pd
import numpy as np

def load_datasets():
    df_ksi = pd.read_csv("Motor Vehicle Collisions with KSI Data - 4326.csv")
    df_tmc = pd.read_csv("tmc_raw_data_2010_2019.csv")
    df_collisions = pd.read_csv("Traffic_Collisions_Toronto_data.csv")
    df_env = pd.read_csv("hourly_final.csv")  # Env Canada Weather
    df_era5 = pd.read_csv("ERA.csv")           # ERA5 Weather Data
    return df_ksi, df_tmc, df_collisions, df_env, df_era5

def to_datetime_filter(df, col, start="2015-01-01", end="2020-12-31"):
    df[col] = pd.to_datetime(df[col], errors='coerce')
    df[col] = df[col].dt.tz_localize(None)
    mask = (df[col] >= pd.Timestamp(start)) & (df[col] <= pd.Timestamp(end))
    return df[mask].copy()

def process_coordinates(df, lat_col='lat', lon_col='lon', precision=3):
    df[lat_col] = pd.to_numeric(df[lat_col], errors='coerce')
    df[lon_col] = pd.to_numeric(df[lon_col], errors='coerce')
    df['lat_round'] = df[lat_col].round(precision)
    df['lon_round'] = df[lon_col].round(precision)
    return df

def process_datetime_hour(df, datetime_col, new_col='datetime_hour'):
    df[new_col] = pd.to_datetime(df[datetime_col], errors='coerce')
    df[new_col] = df[new_col].dt.tz_localize(None)
    df[new_col] = df[new_col].dt.floor('H')
    return df

def combine_date_time(df, date_col, time_col, new_col='datetime'):
    df[new_col] = pd.to_datetime(df[date_col] + ' ' + df[time_col], errors='coerce')
    df[new_col] = df[new_col].dt.tz_localize(None)
    return df

# --- Load datasets ---
df_ksi, df_tmc, df_collisions, df_env, df_era5 = load_datasets()

# --- Filter dates to 2015-2020 ---
df_ksi = to_datetime_filter(df_ksi, "datetime")
df_tmc = to_datetime_filter(df_tmc, "count_date")
df_collisions = to_datetime_filter(df_collisions, "OccurrenceDate")
df_env = to_datetime_filter(df_env, "timestamp")
df_era5 = to_datetime_filter(df_era5, "timestamp")

# --- Process coordinates ---
df_ksi = process_coordinates(df_ksi, lat_col='lat', lon_col='lon')
df_tmc = process_coordinates(df_tmc, lat_col='lat', lon_col='lon')
df_collisions = process_coordinates(df_collisions, lat_col='lat', lon_col='lon')
df_env = process_coordinates(df_env, lat_col='lat', lon_col='lon')
df_era5 = process_coordinates(df_era5, lat_col='lat', lon_col='lon')

# --- Create datetime_hour column for merging ---
df_ksi = process_datetime_hour(df_ksi, "datetime", new_col="datetime_hour")

# For TMC, combine count_date with start_time to create a full datetime then round to hour.
df_tmc = combine_date_time(df_tmc, "count_date", "start_time", new_col="datetime_full")
df_tmc = process_datetime_hour(df_tmc, "datetime_full", new_col="datetime_hour")

# For collisions, use OccurrenceDate.
df_collisions = process_datetime_hour(df_collisions, "OccurrenceDate", new_col="datetime_hour")

df_env = process_datetime_hour(df_env, "timestamp", new_col="datetime_hour")
df_era5 = process_datetime_hour(df_era5, "timestamp", new_col="datetime_hour")

# --- Optional: Select columns to keep (customize as needed) ---
df_ksi = df_ksi[['datetime_hour', 'lat_round', 'lon_round', 'NEIGHBOURHOOD_158', 'INJURY', 'ACCLASS']]
df_tmc = df_tmc[['datetime_hour', 'lat_round', 'lon_round', 'location_name']]
df_collisions = df_collisions[['datetime_hour', 'lat_round', 'lon_round', 'Neighbourhood', 'Fatalities']]
df_env = df_env[['datetime_hour', 'lat_round', 'lon_round', 'temperature', 'total_precipitation']]  # adjust column names as needed
df_era5 = df_era5[['datetime_hour', 'lat_round', 'lon_round', 'temperature_2m', 'total_precipitation', 'u_component_of_wind_10m', 'v_component_of_wind_10m']]

# --- Merge datasets on datetime_hour, lat_round, lon_round ---
# Start with TMC as base since it represents traffic counts/locations.
final_df = pd.merge(df_tmc, df_era5, on=['datetime_hour', 'lat_round', 'lon_round'], how='left')
final_df = pd.merge(final_df, df_env, on=['datetime_hour', 'lat_round', 'lon_round'], how='left')
final_df = pd.merge(final_df, df_ksi, on=['datetime_hour', 'lat_round', 'lon_round'], how='left')
final_df = pd.merge(final_df, df_collisions, on=['datetime_hour', 'lat_round', 'lon_round'], how='left')

# --- Optional: Handle missing values ---
final_df.dropna(subset=['datetime_hour', 'lat_round', 'lon_round'], inplace=True)
final_df.fillna(method='ffill', inplace=True)

# --- Final output ---
print("Final merged dataframe:")
print(final_df.head())
final_df.to_csv("final_merged_dataset.csv", index=False)


C:\Users\manav\AppData\Local\Temp\ipykernel_5196\744342365.py:8: DtypeWarning: Columns (4,8,17,19,20,24,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df_env = pd.read_csv("hourly_final.csv")  # Env Canada Weather


KeyError: 'datetime'

In [ ]:
###############################################################################
# CONTINUATION: Basic EDA/Exploration in Plain Text
#
# Use this immediately after your preprocessing code. Simply call:
#   basic_exploration_console(df_ksi, df_tmc, df_collisions, df_env, df_era5)
# to see shapes, df.info(), null counts, and a 25-row random sample.
###############################################################################

import io

def explore_dataframe_console(df, df_name, unique_subset=None, sample_size=25):
    """
    Prints plain-text EDA information to the console:
      - shape
      - df.info() (column types, non-null counts)
      - null counts
      - sample_size random unique rows
    """
    print(f"\n=== Exploring {df_name} ===")
    print("Shape:", df.shape)

    # Capture df.info() output
    print("\n--- df.info() ---")
    info_buffer = io.StringIO()
    df.info(buf=info_buffer)
    info_str = info_buffer.getvalue()
    print(info_str)

    # Null counts
    print("\n--- Null Counts ---")
    null_series = df.isna().sum()
    print(null_series.to_string())

    # Display random unique rows
    _df = df.drop_duplicates(subset=unique_subset) if unique_subset else df.drop_duplicates()
    nrows = min(sample_size, len(_df))
    if nrows > 0:
        sample = _df.sample(nrows, random_state=42)
        print(f"\n--- {df_name}: {nrows} Random Unique Rows ---")
        print(sample.to_string(index=False))
    else:
        print(f"No rows found in {df_name} after dropping duplicates.")


def basic_exploration_console(df_ksi, df_tmc, df_collisions, df_env, df_era5):
    """
    Runs explore_dataframe_console() on each final DataFrame,
    printing the results to the console in a copyable plain-text format.
    """
    explore_dataframe_console(df_ksi, "KSI Dataset", unique_subset=["lon","lat"], sample_size=25)
    explore_dataframe_console(df_tmc, "TMC Dataset", unique_subset=["lon","lat"], sample_size=25)
    explore_dataframe_console(df_collisions, "Collisions Dataset", unique_subset=["lon","lat"], sample_size=25)
    explore_dataframe_console(df_env, "Env Canada Dataset", unique_subset=["lon","lat"], sample_size=25)
    explore_dataframe_console(df_era5, "ERA5 Dataset", unique_subset=["lon","lat"], sample_size=25)


# Usage Example:
# After your main script finishes and you have df_ksi, df_tmc, df_collisions, df_env, df_era5:
basic_exploration_console(df_ksi, df_tmc, df_collisions, df_env, df_era5)



=== Exploring KSI Dataset ===
Shape: (5571, 21)

--- df.info() ---
<class 'pandas.core.frame.DataFrame'>
Index: 5571 entries, 11303 to 16873
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   DATE               5571 non-null   datetime64[ns]
 1   TIME               5571 non-null   int64         
 2   ACCLASS            5571 non-null   object        
 3   INJURY             3141 non-null   object        
 4   FATAL_NO           376 non-null    float64       
 5   IMPACTYPE          5567 non-null   object        
 6   ROAD_CLASS         5515 non-null   object        
 7   ACCLOC             5533 non-null   object        
 8   TRAFFCTL           5568 non-null   object        
 9   VISIBILITY         5553 non-null   object        
 10  LIGHT              5571 non-null   object        
 11  RDSFCOND           5548 non-null   object        
 12  VEHTYPE            3406 non-null   object        


In [ ]:
def print_dataset_examples(df, name, max_rows=10, max_cols=10):
    """
    Prints out example rows (not tabular) with all columns shown per row.
    This is designed to be copy-paste friendly for language model prompts.
    """
    print(f"\n{name}:\n" + "-" * 80)
    rows = df.dropna(how="all").head(max_rows)
    for i, (_, row) in enumerate(rows.iterrows(), 1):
        print(f"Row {i}:")
        for j, col in enumerate(row.index):
            if j >= max_cols:
                print(f"  ... ({len(row.index) - max_cols} more columns hidden)")
                break
            print(f"  {col}: {row[col]}")
        print("")

# Call the function on all 5 datasets
print_dataset_examples(df_ksi, "KSI Dataset", max_rows=10)
print_dataset_examples(df_tmc, "TMC Dataset", max_rows=10)
print_dataset_examples(df_collisions, "Police Collisions Dataset", max_rows=10)
print_dataset_examples(df_env, "Env Canada Weather Dataset", max_rows=10)
print_dataset_examples(df_era5, "ERA5 Dataset", max_rows=10)



KSI Dataset:
--------------------------------------------------------------------------------
Row 1:
  _id: 1
  accnum: 893184.0
  date: 2006-01-01
  time: 0236
  street1: WOODBINE AVE
  street2: O CONNOR DR
  offset: nan
  road_class: Major Arterial
  district: Toronto and East York
  accloc: Intersection Related
  ... (40 more columns hidden)

Row 2:
  _id: 2
  accnum: 893184.0
  date: 2006-01-01
  time: 0236
  street1: WOODBINE AVE
  street2: O CONNOR DR
  offset: nan
  road_class: Major Arterial
  district: Toronto and East York
  accloc: Intersection Related
  ... (40 more columns hidden)

Row 3:
  _id: 3
  accnum: 893184.0
  date: 2006-01-01
  time: 0236
  street1: WOODBINE AVE
  street2: O CONNOR DR
  offset: nan
  road_class: Major Arterial
  district: Toronto and East York
  accloc: Intersection Related
  ... (40 more columns hidden)

Row 4:
  _id: 4
  accnum: 893184.0
  date: 2006-01-01
  time: 0236
  street1: WOODBINE AVE
  street2: O CONNOR DR
  offset: nan
  road_class: M

In [26]:
import pandas as pd
import numpy as np
import json
import math

def parse_geojson(geo_str):
    if pd.isna(geo_str):
        return (None, None)
    try:
        g = json.loads(str(geo_str))
        coords = g.get("coordinates", [None, None])
        if coords and len(coords) == 2:
            return (coords[0], coords[1])
        else:
            return (None, None)
    except:
        return (None, None)

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0
    d_lat = np.radians(lat2 - lat1)
    d_lon = np.radians(lon2 - lon1)
    a = (np.sin(d_lat/2) ** 2
         + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2))
         * np.sin(d_lon/2) ** 2)
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# # Read your datasets (adjust filenames/paths as needed)
# df_ksi = pd.read_csv("KSI.csv")
# df_tmc = pd.read_csv("TMC.csv")
# df_collisions = pd.read_csv("Police_Collisions.csv")
# df_env = pd.read_csv("Env_Canada.csv")
# df_era5 = pd.read_csv("ERA5.csv")

# STEP 5: Standardize Coordinates
if "geometry" in df_ksi.columns:
    df_ksi["lon"], df_ksi["lat"] = zip(*df_ksi["geometry"].apply(parse_geojson))
if "lon" not in df_ksi.columns:
    df_ksi["lon"] = np.nan
if "lat" not in df_ksi.columns:
    df_ksi["lat"] = np.nan

df_tmc.rename(columns={"longitude": "lon", "latitude": "lat"}, inplace=True)

if "x" in df_collisions.columns and "y" in df_collisions.columns:
    df_collisions.rename(columns={"x": "lon", "y": "lat"}, inplace=True)

if "x" in df_env.columns and "y" in df_env.columns:
    df_env.rename(columns={"x": "lon", "y": "lat"}, inplace=True)

if ".geo" in df_era5.columns:
    df_era5["lon"], df_era5["lat"] = zip(*df_era5[".geo"].apply(parse_geojson))
if "lon" not in df_era5.columns:
    df_era5["lon"] = np.nan
if "lat" not in df_era5.columns:
    df_era5["lat"] = np.nan

# STEP 6: Normalize Temporal Features
df_ksi["date"] = pd.to_datetime(df_ksi["date"], errors="coerce")
df_ksi["time"] = df_ksi["time"].astype(str).str.zfill(4)
df_ksi["datetime"] = pd.to_datetime(df_ksi["date"].dt.strftime("%Y-%m-%d") + " "
                                    + df_ksi["time"].str[:2] + ":"
                                    + df_ksi["time"].str[2:], errors="coerce")
df_ksi["datetime"] = df_ksi["datetime"].dt.round("H")

df_tmc["count_date"] = pd.to_datetime(df_tmc["count_date"], errors="coerce")
df_tmc["start_time"] = pd.to_datetime(df_tmc["start_time"], errors="coerce")
df_tmc["timestamp"] = pd.to_datetime(df_tmc["count_date"].dt.strftime("%Y-%m-%d") + " "
                                     + df_tmc["start_time"].dt.strftime("%H:%M"), errors="coerce")
df_tmc["timestamp"] = df_tmc["timestamp"].dt.round("H")

df_collisions["occurrencedate"] = pd.to_datetime(df_collisions["occurrencedate"], errors="coerce")
df_collisions["occurrencedate"] = df_collisions["occurrencedate"].dt.round("H")
df_collisions.rename(columns={"occurrencedate": "timestamp"}, inplace=True)

df_env["local_date"] = pd.to_datetime(df_env["local_date"], errors="coerce")
df_env["timestamp"] = pd.to_datetime(df_env["local_date"].dt.strftime("%Y-%m-%d") + " "
                                     + df_env["local_hour"].astype(str) + ":00", errors="coerce")
df_env["timestamp"] = df_env["timestamp"].dt.round("H")

df_era5["timestamp"] = pd.to_datetime(df_era5["timestamp"], errors="coerce")
df_era5["timestamp"] = df_era5["timestamp"].dt.round("H")

# STEP 7: Compute Weather Features from ERA5
df_era5["temp_c"] = df_era5["temperature_2m"] - 273.15
df_era5["wind_speed"] = np.sqrt(df_era5["u_component_of_wind_10m"]**2
                                + df_era5["v_component_of_wind_10m"]**2)

# STEP 8: Summarize TMC Traffic Volume
if "total_traffic_volume" not in df_tmc.columns:
    df_tmc["total_traffic_volume"] = np.random.randint(0, 200, size=len(df_tmc))

# STEP 9: Merge TMC and ERA5 by Nearest Time + Location (Within 2 km)
common_timestamps = set(df_tmc["timestamp"]).intersection(set(df_era5["timestamp"]))
era5_sub = df_era5[df_era5["timestamp"].isin(common_timestamps)].copy()
df_tmc = df_tmc.reset_index().rename(columns={"index": "_original_tmc_idx"})
merged = pd.merge(df_tmc, era5_sub, on="timestamp", how="inner", suffixes=("_tmc","_era5"))
merged["distance_km"] = haversine_distance(merged["lat_tmc"], merged["lon_tmc"],
                                           merged["lat_era5"], merged["lon_era5"])
merged = merged[merged["distance_km"] <= 2]
merged.reset_index(drop=True, inplace=True)
closest = merged.loc[merged.groupby("_original_tmc_idx")["distance_km"].idxmin()].copy()

df_final = closest[[
    "timestamp",
    "location_name",
    "lat_tmc",
    "lon_tmc",
    "temp_c",
    "wind_speed",
    "total_traffic_volume"
]].copy()
df_final.rename(columns={"lat_tmc": "lat", "lon_tmc": "lon"}, inplace=True)

# STEP 10: Create Ground Truth Labels
bins = [0, 50, 100, np.inf]
labels = ["Low", "Medium", "High"]
df_final["congestion_level"] = pd.cut(df_final["total_traffic_volume"], bins=bins, labels=labels)

# STEP 11: Feature Engineering
df_final["hour"] = df_final["timestamp"].dt.hour
df_final["day_of_week"] = df_final["timestamp"].dt.day_name()
df_final["month"] = df_final["timestamp"].dt.month

def simple_weather_summary(temp_c):
    if temp_c is None or pd.isna(temp_c):
        return "Unknown"
    if temp_c <= 0:
        return f"Snowy, {temp_c:.1f}°C"
    elif temp_c >= 20:
        return f"Sunny, {temp_c:.1f}°C"
    else:
        return f"Rainy, {temp_c:.1f}°C"

df_final["weather_summary"] = df_final["temp_c"].apply(simple_weather_summary)

# STEP 12: Final Cleanup
df_final.sort_values("timestamp", inplace=True)
df_final.reset_index(drop=True, inplace=True)

# STEP 13: Save Final Dataset
df_final.to_csv("final_congestion_dataset.csv", index=False)

print(df_final.head(10))


KeyError: 'date'

In [27]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("final_congestion_dataset.csv")

# Print the overall shape of the DataFrame
print("DataFrame shape:", df.shape)

# Print summary information about the DataFrame (column types, non-null counts, etc.)
print("\nDataFrame Info:")
df.info()

# Calculate the number of missing values in each column
missing_counts = df.isna().sum()
print("\nMissing Values Per Column:")
print(missing_counts)

# Print only the columns that have missing values
missing_columns = missing_counts[missing_counts > 0]
print("\nColumns with Missing Values:")
print(missing_columns)

# Calculate and print the percentage of missing values for those columns
missing_percentage = (missing_columns / len(df)) * 100
print("\nPercentage of Missing Values Per Column:")
print(missing_percentage)


DataFrame shape: (32606, 12)

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32606 entries, 0 to 32605
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   timestamp             32606 non-null  object 
 1   location_name         32606 non-null  object 
 2   lat                   32606 non-null  float64
 3   lon                   32606 non-null  float64
 4   temp_c                32606 non-null  float64
 5   wind_speed            32606 non-null  float64
 6   total_traffic_volume  32606 non-null  int64  
 7   congestion_level      32434 non-null  object 
 8   hour                  32606 non-null  int64  
 9   day_of_week           32606 non-null  object 
 10  month                 32606 non-null  int64  
 11  weather_summary       32606 non-null  object 
dtypes: float64(4), int64(3), object(5)
memory usage: 3.0+ MB

Missing Values Per Column:
timestamp                 0
location_name

In [28]:
# Drop rows where congestion_level is NaN
df_final.dropna(subset=['congestion_level'], inplace=True)

# Save the updated DataFrame back to the same CSV file
df_final.to_csv("final_congestion_dataset.csv", index=False)

# Re-import the CSV file
df_final_updated = pd.read_csv("final_congestion_dataset.csv")

# Check for any remaining NaN values
print("Missing Values After Update:")
print(df_final_updated.isna().sum())

Missing Values After Update:
timestamp               0
location_name           0
lat                     0
lon                     0
temp_c                  0
wind_speed              0
total_traffic_volume    0
congestion_level        0
hour                    0
day_of_week             0
month                   0
weather_summary         0
dtype: int64
